/* PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  

QUESTIONS:
/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */


/* Q12: Find the facilities with their usage by member, but not guests */


/* Q13: Find the facilities usage by month, but not guests */



In [63]:
from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy import inspect
from sqlalchemy import text


In [57]:
engine = create_engine("sqlite:///sqlite_db_pythonsqlite.db")

In [58]:
type(engine)

sqlalchemy.engine.base.Engine

In [60]:
insp = inspect(engine)
print(insp.get_table_names())

['Bookings', 'Facilities', 'Members']


* Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members! */

In [74]:
with engine.connect() as con:
    rs = con.execute(text("SELECT facid, SUM(Cost) AS Revenue \
                          FROM (SELECT Bookings.facid, memid, slots, membercost, guestcost, \
                           CASE WHEN memid = 0 THEN slots*guestcost \
                              ELSE slots * membercost END AS Cost   \
                          FROM Bookings \
                          LEFT JOIN Facilities \
                          ON Bookings.facid = Facilities.facid) AS subquerry \
                          GROUP BY facid \
                          ORDER BY Revenue;"))
    df = pd.DataFrame(rs.fetchall())
    print(df)

   facid  Revenue
0      3    180.0
1      7    240.0
2      8    270.0
3      2   1906.5
4      6  13468.0
5      0  13860.0
6      1  14310.0
7      5  14454.6
8      4  50351.6


/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

In [102]:
with engine.connect() as con:
    rs = con.execute(text("SELECT e.surname, e.firstname,m.firstname || ' ' || m.surname AS Recommendedby \
                            FROM Members e \
                            LEFT JOIN Members m \
                            ON e.recommendedby = m.memid \
                            WHERE e.recommendedby <> '';"))
    df = pd.DataFrame(rs.fetchall())
    print(df)

              surname  firstname      Recommendedby
0            Joplette     Janice       Darren Smith
1             Butters     Gerald       Darren Smith
2                Dare      Nancy    Janice Joplette
3              Boothe        Tim         Tim Rownam
4            Stibbons     Ponder       Burton Tracy
5                Owen    Charles       Darren Smith
6               Jones      David    Janice Joplette
7               Baker       Anne    Ponder Stibbons
8               Smith       Jack       Darren Smith
9               Bader   Florence    Ponder Stibbons
10              Baker    Timothy     Jemima Farrell
11             Pinker      David     Jemima Farrell
12            Genting    Matthew     Gerald Butters
13          Mackenzie       Anna       Darren Smith
14             Coplin       Joan      Timothy Baker
15             Sarwin  Ramnaresh     Florence Bader
16              Jones    Douglas        David Jones
17             Rumney  Henrietta    Matthew Genting
18  Worthing

/* Q12: Find the facilities with their usage by member, but not guests */

In [113]:
with engine.connect() as con:
    rs = con.execute(text("SELECT Facilities.name, subquerry.usage \
                        FROM ( \
                        SELECT SUM(slots) AS \"usage\", facid \
                        FROM Bookings \
                        WHERE memid !=0 \
                        GROUP BY facid \
                        ) subquerry \
                        LEFT JOIN Facilities \
                        ON subquerry.facid = Facilities.facid \
                        ORDER BY usage;"))
    df = pd.DataFrame(rs.fetchall())
    print(df)

              name  usage
0   Massage Room 2     54
1     Squash Court    418
2     Table Tennis    794
3       Pool Table    856
4    Snooker Table    860
5   Tennis Court 2    882
6   Massage Room 1    884
7   Tennis Court 1    957
8  Badminton Court   1086


/* Q13: Find the facilities usage by month, but not guests */

In [131]:
with engine.connect() as con:
    rs = con.execute(text(" \
                        SELECT Facilities.name, MONTH, USAGE \
                        FROM \
                        (SELECT strftime('%m', starttime) AS \"MONTH\", facid, SUM(slots) AS \"USAGE\" \
                        FROM Bookings \
                        WHERE memid !=0 \
                        GROUP BY facid, MONTH ) subquerry\
                        LEFT JOIN Facilities \
                        ON subquerry.facid = Facilities.facid \
                    ;"))
    df = pd.DataFrame(rs.fetchall())
    print(df)

               name MONTH  USAGE
0    Tennis Court 1    07    201
1    Tennis Court 1    08    339
2    Tennis Court 1    09    417
3    Tennis Court 2    07    123
4    Tennis Court 2    08    345
5    Tennis Court 2    09    414
6   Badminton Court    07    165
7   Badminton Court    08    414
8   Badminton Court    09    507
9      Table Tennis    07     98
10     Table Tennis    08    296
11     Table Tennis    09    400
12   Massage Room 1    07    166
13   Massage Room 1    08    316
14   Massage Room 1    09    402
15   Massage Room 2    07      8
16   Massage Room 2    08     18
17   Massage Room 2    09     28
18     Squash Court    07     50
19     Squash Court    08    184
20     Squash Court    09    184
21    Snooker Table    07    140
22    Snooker Table    08    316
23    Snooker Table    09    404
24       Pool Table    07    110
25       Pool Table    08    303
26       Pool Table    09    443
